In [1]:
#extracts comments in Json format
from config import API_TOKEN
import requests
import json
# Jira API endpoint
JIRA_URL = "https://Hibernate.atlassian.net/rest/api/3"

# Authentication credentials
USERNAME = "omar.khabthani123@gmail.com"
API_TOKEN = API_TOKEN
def fetch_issues(project_key):
    auth = (USERNAME, API_TOKEN)
    response = requests.get(
        f"{JIRA_URL}/search",
        auth=auth,
        params={"jql": f"project={project_key}"}
    )
    response.raise_for_status()
    return response.json()["issues"]

def fetch_comments(issue_key):
    auth = (USERNAME, API_TOKEN)
    response = requests.get(
        f"{JIRA_URL}/issue/{issue_key}/comment",
        auth=auth
    )
    response.raise_for_status()
    return response.json()["comments"]

def extract_comments_for_project(project_key):
    issues = fetch_issues(project_key)
    
    issue_comments = {}
    for issue in issues:
        comments = fetch_comments(issue["key"])
        issue_comments[issue["key"]] = comments
    
    return issue_comments

def save_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

if __name__ == "__main__":
    project_key = "HHH"
    issue_comments = extract_comments_for_project(project_key)
    json_filename = "issue_comments.json"
    save_to_json(issue_comments, json_filename)
    print(f"Data saved to {json_filename}")

Data saved to issue_comments.json


In [17]:
#extract comment in html
from config import API_TOKEN
import requests
from requests.auth import HTTPBasicAuth
import json

url = "https://hibernate.atlassian.net/rest/api/3/issue/HHH-17757/comment?expand=renderedBody"

auth = HTTPBasicAuth("omar.khabthani123@gmail.com", API_TOKEN)
headers = {
  "Accept": "application/json"
}

response = requests.request(
   "GET",
   url,
   headers=headers,
   auth=auth,
)

print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))


{
    "comments": [
        {
            "author": {
                "accountId": "557058:c227967f-4207-4715-82ea-1c48a845aea1",
                "accountType": "atlassian",
                "active": true,
                "avatarUrls": {
                    "16x16": "https://secure.gravatar.com/avatar/61a63fed50c84da73ec134c652cdfd27?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%2Finitials%2FGK-1.png",
                    "24x24": "https://secure.gravatar.com/avatar/61a63fed50c84da73ec134c652cdfd27?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%2Finitials%2FGK-1.png",
                    "32x32": "https://secure.gravatar.com/avatar/61a63fed50c84da73ec134c652cdfd27?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%2Finitials%2FGK-1.png",
                    "48x48": "https://secure.gravatar.com/avatar/61a63fed50c84da73ec134c652cdfd27?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public